# 基于 ERNIE-4.5-0.3B 的大语言模型直接偏好优化 (DPO) 教程

## 1. 引言

欢迎来到这篇关于使用 ERNIE Kit 对 ERNIE-4.5-0.3B 模型进行直接偏好优化 (Direct Preference Optimization, DPO) 的高级教程！在完成了模型的预训练 (PT) 和监督微调 (SFT) 之后，我们已经拥有了一个具备强大通用语言能力且能够精准遵循指令的 ERNIE 模型。然而，为了让模型生成的回答不仅正确，更能符合人类细致入微的偏好——例如，更具帮助性、更安全无害、风格更受欢迎——我们通常需要进行进一步的“对齐”(Alignment) 训练。本教程将深入探讨如何通过 DPO 技术，让您的 ERNIE 模型实现更高层次的对齐。

### 1.1 什么是直接偏好优化 (DPO)？

**直接偏好优化 (DPO)** 是一种革命性的、相较于传统方法更简单高效的模型对齐技术。在 DPO 出现之前，业界主流的对齐方法是基于人类反馈的强化学习 (Reinforcement Learning from Human Feedback, RLHF)，该方法通常涉及多个复杂且资源密集型的阶段：

1.  **监督微调 (SFT)** ：首先，对预训练好的大语言模型进行指令微调，使其学会理解并执行各种指令，这是对齐的第一步，赋予模型基础的“能力”。
2.  **训练奖励模型 (Reward Model, RM)** ：接下来，需要收集大量人类对模型多种输出的偏好排序数据。例如，针对同一个问题，模型生成了回答A、B、C，标注者会指出哪个最好、哪个次之、哪个最差。利用这些排序数据，训练一个独立的“奖励模型”，这个模型的任务是模仿人类的判断力，为任何一个给定的回答打分，分数高低代表了人类的偏好程度。
3.  **强化学习 (RL)** ：最后，将 SFT 阶段的模型作为“智能体”，在奖励模型的指导下进行强化学习训练（通常使用近端策略优化 PPO 算法）。智能体尝试生成新的回答，奖励模型则对其进行打分，智能体根据得分不断调整自身策略，以期获得更高的奖励，从而使其输出越来越符合人类偏好。

尽管 RLHF 被证明非常有效，但其流程冗长、实现复杂、训练过程不稳定，并且需要同时维护和协调多个模型（策略模型、奖励模型、价值模型、参考模型等），对技术和计算资源都提出了很高的要求。

**DPO 的核心思想在于其“直接性”——它巧妙地绕过了显式训练奖励模型和复杂的强化学习步骤。** DPO 认为，语言模型本身就可以隐式地代表一个奖励函数。它直接利用成对的人类偏好数据（例如，给定一个提示，回答A比回答B更好），通过设计一个独特的损失函数，直接在语言模型上进行优化。这个损失函数的目标是，让模型提高其生成“偏好”回答的概率，同时降低其生成“不偏好”回答的概率。本质上，DPO 将复杂的强化学习问题转化为了一个更易于处理的、带有特定目标的有监督学习问题。

**DPO 与 RLHF/SFT 的核心区别对比：**

| 特性 | 监督微调 (SFT) | 传统 RLHF (RM+PPO) | 直接偏好优化 (DPO) |
| :--- | :--- | :--- | :--- |
| **核心目标** | 学习遵循指令，掌握特定任务的解决方法 | 通过奖励模型信号，学习生成符合人类偏好的回答 | 直接基于偏好数据，学习生成符合人类偏好的回答 |
| **所需数据** | “指令-理想输出”对 | “指令-多个输出-人类排序” (用于RM)，以及SFT数据 | “指令-偏好输出-非偏好输出”三元组 |
| **学习范式** | 有监督学习 | RM:有监督学习; PPO:强化学习 | 有监督学习 (通过特殊损失函数体现偏好) |
| **实现复杂性** | 相对简单、直接 | 非常高 (多阶段训练，涉及复杂的RL算法和超参) | 中等 (单阶段策略学习，但数据格式要求特定) |
| **所需模型数量** | 1 (策略模型) | 至少2个核心模型 (策略模型, 奖励模型)，常伴有价值模型和参考模型 | 1个核心模型 (策略模型)，通常会用到一个冻结的SFT模型作为参考 |

简而言之，如果说 SFT 教会了模型“能做什么”，那么 DPO (和 RLHF) 则是教会模型“如何做得更好、更符合人类期望、更安全可靠”。DPO 为实现这一目标提供了一条更直接、更简洁、更稳定的技术路径。

### 1.2 DPO 的应用场景与价值

DPO 作为对齐技术的前沿方法，其应用场景和意义极为广泛和深远，我们可以从以下几个维度来理解：

|  | 应用领域 | 核心价值与深远意义 |
| :---: | :--- | :--- |
| 💡 | **提升回答质量 (Helpfulness)** | 通过精细学习人类偏好，模型能生成更准确、信息量更丰富、逻辑更严谨的回答，从“指令执行者”蜕变为真正的“智能助手”。 |
| 🛡️ | **增强模型安全 (Harmlessness)** | 通过学习无害/有害内容的边界，有效抑制模型生成不当、有偏见或危险的言论，使其输出更符合社会规范和伦理要求。 |
| 🎨 | **精准控制风格 (Style Control)** | 可根据特定偏好数据，引导模型采用特定沟通风格，如客服场景下的礼貌同理、代码场景下的简洁清晰、创作场景下的天马行空。 |
| 🚀 | **简化对齐流程 (Efficiency)** | 作为 RLHF 的高效替代方案，DPO 降低了实现高质量对齐的技术门槛和计算成本，适合资源有限的团队和个人快速迭代。 |
| 🔬 | **驱动前沿创新 (Innovation)** | DPO 的成功激发了 IPO, KTO, SimPO, ORPO 等一系列新算法的诞生，是理解和跟进大模型技术最新发展趋势的基础。 |

### 1.3 为什么选择 ERNIE-4.5-0.3B 进行 DPO？

|  | 理由 | 详细说明 |
| :---: | :--- | :--- |
| 🏆 | **卓越的 SFT 基础** | `ERNIE-4.5-0.3B` 基于文心4.5的先进技术，并经过高质量SFT，具备强大的指令遵循能力和世界知识，为DPO的“画龙点睛”提供了坚实基础。 |
| 💻 | **合适的参数规模** | 0.3B的参数量在保持强大能力的同时，对计算资源要求可控，非常适合个人开发者、学术研究者和中小企业进行DPO实验和快速迭代。 |
| 🛠️ | **ERNIE Kit 全面支持** | ERNIE Kit 提供了从数据处理到训练执行的端到端DPO解决方案 (`run_dpo.py`)，让开发者无需从头实现复杂算法，即可高效应用DPO。 |

### 1.4 本教程的目标与收获

本教程专为希望进阶模型对齐的开发者和研究者设计，通过学习，您将获得：

|  | 学习成果 | 描述 |
| :---: | :--- | :--- |
| 🧠 | **深刻理解 DPO 原理** | 透彻理解 DPO 的核心思想、数学原理、应用价值，以及与 SFT/RLHF 的本质区别。 |
| 🛠️ | **熟练掌握 ERNIE Kit DPO 全流程** | 完整掌握从环境配置、数据准备到模型训练、评估的全过程。 |
| 📊 | **精通 DPO 数据处理** | 学会处理符合 ERNIE Kit 要求的偏好数据格式 (prompt, chosen, rejected)。 |
| ⚙️ | **灵活配置与实践** | 学会修改 DPO 配置文件，启动、监控、调试训练任务，并对优化后的模型进行评估。 |

现在，就让我们一起踏上这段激动人心的旅程，探索如何通过 DPO 技术，将强大的 ERNIE 模型雕琢成一个更加智能、更加安全、更懂人类心意的顶尖 AI 吧！

## 2. 环境准备

进行直接偏好优化 (DPO) 与进行 SFT 类似，都需要一个配置完善的开发环境。这包括安装核心的深度学习框架 PaddlePaddle、专门的 ERNIE Kit 开发套件，并获取其最新的源代码以使用其中最前沿的 DPO 功能。我们假设您已经具备 SFT 的实践基础，因此对环境配置流程会比较熟悉，但本节将提供更详尽的指导，确保万无一失。

### 2.1 安装 PaddlePaddle 和 ERNIE Kit

首先，我们需要确保安装了正确版本的 PaddlePaddle 和 ERNIE Kit。为了获得最佳的训练性能和兼容性，强烈推荐使用 GPU 版本的 PaddlePaddle。

*如果在AI studio运行的同学不需要运行下方代码块*

In [ ]:
# 步骤 1: 确保您的 pip 工具是最新版本，这可以避免很多潜在的安装问题
!python -m pip install --upgrade pip

# 步骤 2: 安装 PaddlePaddle 的 GPU 版本
# 以下命令适用于 CUDA 11.8 或更高版本的环境。如果您的 CUDA 版本不同，
# 请务必访问 PaddlePaddle 官方网站获取完全匹配的安装指令：https://www.paddlepaddle.org.cn/install/quick
!python -m pip install paddlepaddle-gpu -i https://mirror.baidu.com/pypi/simple



In [ ]:
# 步骤 3: 安装 ERNIE Kit
# ERNIE Kit 提供了两种安装方式，我们推荐从源码安装，以确保能使用到最新的 DPO 功能和 bug 修复。
# 首先，需要从 GitHub 克隆 ERNIE Kit 的仓库。
# !git clone https://github.com/PaddlePaddle/ERNIE.git
# 然后，通过 pip 以可编辑模式 (-e) 安装，这样您对代码的任何修改都能立刻生效。
# 请将下面的路径替换为您实际克隆 ERNIE Kit 的路径。
!python -m pip install -e  ./ERNIE-develop

**安装后验证**：

执行以下代码，检查 PaddlePaddle 和 ERNIE Kit 是否成功安装，并确认 GPU 环境是否可用。

In [2]:
import paddle

print(f"PaddlePaddle Version: {paddle.__version__}")

# 运行 PaddlePaddle 的内置检查工具，它会提供详细的环境信息
try:
    paddle.utils.run_check()
    if paddle.device.cuda.device_count() > 0:
        print(f"[SUCCESS] PaddlePaddle GPU is available! Found {paddle.device.cuda.device_count()} GPU(s).")
    else:
        print("[WARNING] PaddlePaddle GPU check passed, but no GPU found. Training will proceed on CPU, which will be very slow.")
except Exception as e:
    print(f"[ERROR] PaddlePaddle GPU check failed: {e}")
    print("If you intended to use GPU, please carefully check your CUDA environment, NVIDIA driver, and the PaddlePaddle version you installed.")

PaddlePaddle Version: 3.1.0
Running verify PaddlePaddle program ... 
PaddlePaddle works well on 1 GPU.
PaddlePaddle is installed successfully! Let's start deep learning with PaddlePaddle now.
[SUCCESS] PaddlePaddle GPU is available! Found 1 GPU(s).


I0718 09:54:48.481559  9362 pir_interpreter.cc:1524] New Executor is Running ...
W0718 09:54:48.483019  9362 gpu_resources.cc:114] Please NOTE: device: 0, GPU Compute Capability: 8.0, Driver API Version: 12.8, Runtime API Version: 12.6
I0718 09:54:48.483794  9362 pir_interpreter.cc:1547] pir interpreter is running by multi-thread mode ...


### 2.2 下载 ERNIE Kit 仓库代码

ERNIE Kit 的核心 DPO 脚本 (`examples/long_text/run_dpo.py` 或类似路径) 和所有相关的模型配置文件都位于其 GitHub 仓库中。即使您通过 pip 安装了 ERNIE Kit，我们也强烈建议您克隆一份完整的代码仓库，因为所有实验和训练都将围绕这份代码进行。

In [4]:
# 如果您在之前的步骤中尚未克隆，请执行此命令
# git clone https://github.com/PaddlePaddle/ERNIE.git

# 克隆后，您会得到一个名为 ERNIE-develop (或 ERNIE) 的文件夹
# 我们后续所有命令的执行，都将假设您位于这个文件夹的根目录下
# 例如: cd ./ERNIE-develop
%cd ./ERNIE-develop

/home/aistudio/ERNIE-develop


## 3. DPO 数据准备

与 SFT 阶段使用“指令-理想输出”对的数据格式截然不同，直接偏好优化 (DPO) 的精髓在于利用人类对模型不同输出的**偏好判断**。因此，DPO 的训练数据直接、精确地体现了这种成对的偏好选择，这也是其“直接”优化的基础。

### 3.1 DPO 数据的特点与格式

DPO 训练数据的基本单元是一个三元组，包含：**“提示 (Prompt)”**、**“被偏好的回答 (Chosen Response)”** 和 **“不被偏好的回答 (Rejected Response)”**。这样的数据结构明确地告诉模型：在接收到同一个提示时，哪一种类型的回答更符合人类的期望，哪一种则不然。

**ERNIE Kit 支持的 DPO 数据格式：**

根据 ERNIE Kit 的设计，DPO 脚本期望的数据是 **JSON Lines (jsonl) 文件**。这是一种非常流行的格式，文件中的每一行都是一个独立的、合法的 JSON 对象，代表一个偏好样本。一个典型的样本结构如下所示：

```json
{
    "prompt": "请向我解释一下什么是人工智能？",
    "chosen": "人工智能（AI）是计算机科学的一个分支，致力于创造能够模仿、扩展和超越人类智能的机器。它涵盖了机器学习、自然语言处理、计算机视觉等多个领域，旨在让计算机能够像人一样学习、推理、感知和解决问题。例如，我们手机上的语音助手、自动驾驶汽车以及推荐系统，都是人工智能在日常生活中的应用。",
    "rejected": "人工智能就是机器人。"
}
```

**字段深度解析**：

*   `prompt`: `(string)`，必需。这是输入给模型的用户指令、问题或前文对话。它是偏好判断的上下文，是所有比较的起点。
*   `chosen`: `(string)`，必需。这是针对 `prompt` 的两个回答中，被人类标注为**更偏好**、**质量更高**的那个。它代表了模型应该学习和模仿的方向。
*   `rejected`: `(string)`，必需。这是针对 `prompt` 的两个回答中，被人类标注为**相对不偏好**、**质量较低**的那个。它代表了模型应该避免和抑制的方向。

**为什么是这种“提示-选择-拒绝”的三元组格式？**

DPO 的损失函数需要直接比较模型对于 `chosen` 回答和 `rejected` 回答分别赋予的概率（或隐式的偏好得分）。因此，每一条训练数据都必须清晰地提供这一对成对比较的 (chosen, rejected) 输出，以及它们所共同对应的输入提示 `prompt`。这种结构使得损失函数可以直接计算偏好差异，并据此更新模型参数。

**高质量DPO数据的关键要素：**

构建或选用一个高质量的DPO数据集是成功的关键。一个好的数据集应具备以下特点：

*   **明确且有意义的偏好差异**：Chosen 和 Rejected 回答之间应该存在清晰、可辨别的质量差异。这种差异应该精确反映你希望模型学习到的特定偏好，例如：
    *   **事实准确性**：Chosen 回答更真实，Rejected 回答包含事实错误。
    *   **无害性**：Chosen 回答安全、礼貌，Rejected 回答包含有害、冒犯性或有偏见的内容。
    *   **帮助性与详细程度**：Chosen 回答更全面、更有帮助，Rejected 回答过于简略或答非所问。
    *   **指令遵循**：Chosen 回答严格遵循了指令中的所有约束（如格式、角色），Rejected 回答则没有。
    *   **风格偏好**：Chosen 回答符合特定的风格（如专业、幽默），Rejected 回答则不符合。
*   **提示的多样性**：`prompt` 应该广泛覆盖模型在实际应用中可能遇到的各种指令类型、主题和难度，以确保模型学到的偏好具有良好的泛化能力。
*   **回答的真实性与挑战性**：`chosen` 和 `rejected` 的回答最好是模型在SFT阶段的真实输出，或者是经过精心构造、能够代表模型典型错误或潜在改进方向的例子。两者差异不应过于极端（例如一个完美答案 vs 一个完全无关的答案），适度的难度更能激发模型的学习潜力。
*   **一致的标注标准**：如果数据由多人标注，必须建立并遵循一套统一、明确的偏好判断标准，以保证数据质量的可靠性。

### 3.2 使用 ERNIE Kit 自带的示例 DPO 数据集

为了方便用户快速上手，ERNIE Kit 项目在 `examples/data/` 目录下已经内置了一份小型的 DPO 示例数据集。这份数据可以直接用于运行教程中的 DPO 训练脚本，无需从网络下载。

**数据集文件：**

*   `examples/data/dpo-train.jsonl`: 用于 DPO 训练的偏好数据集。
*   `examples/data/dpo-eval.jsonl`: 用于在训练过程中评估模型性能的偏好数据集。

这两个文件都遵循我们之前讨论的 JSON Lines 格式，包含了 `prompt`, `chosen`, 和 `rejected` 字段。

**查看数据样本：**

假设您当前的工作目录是 ERNIE Kit 的根目录，您可以使用以下代码来查看训练数据的内容，直观地了解其结构和质量。

In [7]:
import os
import json

# ERNIE Kit 自带的 DPO 训练数据路径
dpo_train_file = "./examples/data/dpo-train.jsonl"

if os.path.exists(dpo_train_file):
    print(f"--- 查看自带的 DPO 训练数据样本: {dpo_train_file} ---")
    with open(dpo_train_file, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i < 3: # 只看前3个样本
                sample = json.loads(line.strip())
                print(f"\n--- 样本 {i+1} ---")
                # 根据文件格式，'src' 是一个列表，我们取第一个元素作为 prompt
                print(f"Prompt: {sample['src'][0]}")
                # 'response' 是一个包含两个回答的列表，'sort' 字段的 [1, 0] 表示第一个回答是 chosen，第二个是 rejected
                print(f"\nChosen: {sample['response'][0]}")
                print(f"\nRejected: {sample['response'][1]}")
                print("-" * 20)
            else:
                break
else:
    print(f"[ERROR] 未找到 DPO 训练数据文件: {dpo_train_file}。请确认您的工作目录是否为 ERNIE Kit 根目录，并且该文件确实存在。")

--- 查看自带的 DPO 训练数据样本: ./examples/data/dpo-train.jsonl ---

--- 样本 1 ---
Prompt: 请写一份关于支撑航空和飞行的物理基本原理的全面解释，包括升力、推力、重量和阻力等主题，以及伯努利定律、牛顿运动定律和空气性质等关键科学概念。使用清晰简洁的语言，并提供详细的例子和插图，以帮助读者理解这些概念。考虑塑造航空业的历史和技术发展，以及物理学在推动飞行能力方面发挥的作用。

Chosen: ['飞行物理学：航空原理全面指南\n\n**介绍**\n\n理解支撑航空和飞行的物理原理对于理解飞机在空中飞行的机制至关重要。在本指南中，我们将探讨飞行的基本因素，包括升力、推力、重量和阻力，并探索关键的科学概念，如伯努利定理、牛顿运动定律和空气的特性。通过使用清晰、简明的语言，并提供详细的例子和插图，本指南旨在使航空和飞行的概念对读者易于理解和吸引。\n\n**1. 飞行的四个力量**\n\n首先，认识到在飞行过程中作用于飞机的四个关键力量是很重要的：升力、推力、重量和阻力。这些力量之间的微妙平衡是飞机保持空中飞行和维持其所需飞行路径所必需的。\n\n1.1. 升力：升力是抵抗飞机重量并支撑其在空中的力量。升力是通过操纵飞机机翼周围的气压分布来产生的。机翼的形状，称为翼型，是为了优化升力而设计的。升力主要是通过伯努利定理和牛顿运动定律来解释的，我们稍后将详细探讨。\n\n1.2. 推力：推力是由飞机发动机产生的向前的力量，推动飞机穿过空气。喷气发动机、螺旋桨发动机甚至火箭发动机都可以提供必要的推力来克服阻力，这是另一个控制飞行的关键因素。\n\n1.3. 重量：重量是由于飞机质量所产生的重力。它向下作用并抵抗升力。为了保持水平飞行，飞机必须产生足够的升力来抵消其重量。\n\n1.4. 阻力：阻力是阻碍飞机在空气中运动的力量。它主要由两种类型组成：形状阻力，由于飞机的形状而产生，以及皮肤摩擦阻力，由于空气与飞机表面之间的摩擦而产生。最小化阻力对于实现高效飞行至关重要。\n\n**2. 伯努利定理和升力**\n\n伯努利定理是流体动力学中的基本概念，对于理解升力至关重要。该定理指出，当流体（在这种情况下是空气）的速度增加时，其压力降低。飞机机翼采用翼型形状，使空气在机翼上方移动得比下方快。这导致上表面的压力降低，下表

通过运行以上代码，您可以清晰地看到 DPO 数据的具体形式，这对于理解后续的训练过程至关重要。现在，数据已经准备就绪，我们可以继续探索 DPO 训练的核心环节了。

通过打印出的样本，您可以清晰地看到每个数据点是如何由 `prompt`、`chosen` 和 `rejected` 构成的，这有助于您更好地理解 DPO 的训练机制，并为您将来构建自己的数据集提供参考。

### 3.3 数据预处理与Tokenization核心要点

与 SFT 和预训练阶段类似，DPO 训练前也必须将文本数据转换为模型能够理解的数字序列（Token IDs）。ERNIE Kit 的 `run_dpo.py` 脚本在后台会自动处理复杂的预处理逻辑，但理解其核心机制至关重要：

1.  **Tokenization**：脚本会使用与您的基础模型（ERNIE-4.5-0.3B，通常是SFT后的版本）完全匹配的 Tokenizer。对于每个样本，`prompt`、`chosen` 和 `rejected` 这三段文本都会被独立地进行 Tokenize。

2.  **序列构建与损失计算**：
    *   DPO 的核心是比较模型对 `chosen` 和 `rejected` 回答的偏好程度。这通常通过计算模型在这两个回答序列上的**对数概率 (log-probabilities)** 来实现。
    *   为了计算这些概率，脚本会构建两个输入序列：一个是 `prompt` 与 `chosen` 的拼接 (`prompt + chosen`)，另一个是 `prompt` 与 `rejected` 的拼接 (`prompt + rejected`)。
    *   **标签 (Labels) 的智能处理**：在计算损失时，我们只关心模型对**回答部分**的预测能力，而 `prompt` 部分只是上下文，不应计入损失。这是通过在 `labels` 中对非回答部分的 token 进行**掩码 (masking)** 来实现的（通常设置为-100）。
        *   对于 `prompt + chosen` 序列，其对应的 `labels` 中，`prompt` 部分被 mask 掉，只有 `chosen` 部分是真实的 token ID。
        *   同理，对于 `prompt + rejected` 序列，其 `labels` 中，`prompt` 部分被 mask 掉，只有 `rejected` 部分是真实的 token ID。
    *   **DPO 损失函数**：最终，DPO 损失函数会利用模型在这两组序列上计算出的对数概率之差，来引导模型参数的更新，目标是让 `chosen` 回答的概率相对 `rejected` 回答的概率更高。

3.  **长度控制：截断与填充**：
    *   实际应用中，文本长度各不相同。脚本会使用 `max_length` (或类似的) 参数来统一处理。过长的序列（`prompt + response`）会被截断，不足的则会用 padding token 填充到相同长度，以便进行批处理训练。

4.  **参考模型 (Reference Model) 的作用**：
    *   标准的 DPO 算法会引入一个“参考模型”，它通常就是 SFT 阶段结束后的那个模型，并且在 DPO 训练过程中其参数**完全冻结，不进行更新**。
    *   DPO 损失函数在计算 `chosen` 和 `rejected` 回答的对数概率时，会减去参考模型在相同回答上的对数概率。这可以被理解为一种**正则化**手段，目的是防止策略模型（即我们正在训练的模型）为了过度迎合偏好数据而偏离其在 SFT 阶段学到的通用语言能力太远，从而导致“能力灾难性遗忘”。它确保模型在学习“偏好”的同时，不会丢掉“能说人话”的基础。

幸运的是，作为使用者，我们通常只需要在配置文件中正确指定数据集路径、SFT模型路径、以及相关的序列长度和 DPO 核心超参数（如 `beta`）。ERNIE Kit 的 `run_dpo.py` 脚本会为我们妥善处理上述所有复杂的 Tokenization、序列拼接、标签掩码和损失计算细节。

至此，我们已经拥有了高质量、格式正确的偏好数据集，并深刻理解了其背后的处理逻辑。这是 DPO 训练成功的坚实基石。

### 3.1 ERNIE Kit DPO 数据格式

根据我们对 `ERNIE-develop` 项目的分析，ERNIE Kit 的DPO脚本期望的数据是 **JSON Lines (jsonl) 文件**，其中每一行是一个JSON对象，代表一个偏好样本。一个典型的样本结构如下：

```json
{
    "system": "系统提示词",
    "src": ["用户问题1", "用户问题2"],
    "tgt": ["模型在遇到用户问题1时的回答"],
    "response": [
        ["针对用户问题2的偏好回答"],
        ["针对用户问题2的不偏好回答"]
    ],
    "sort": [1, 0]
}
```

**字段解释**：

*   `system`: (可选) `str`。系统级的提示词，用于设定模型的角色或行为准则。
*   `src`: `List[str]`。用户的输入序列，在多轮对话中，这会是一个包含多条用户输入的列表。
*   `tgt`: `List[str]`。模型在前几轮对话中的回答。
*   `response`: `List[List[str]]`。一个包含两个列表的列表。第一个列表是**chosen response**（偏好回答），第二个列表是**rejected response**（非偏好回答）。
*   `sort`: `List[int]`。一个包含两个整数的列表，`[1, 0]` 表示 `response` 中第一个回答是偏好的，第二个是不偏好的。

ERNIE Kit 提供了一个示例数据集，我们可以参考其格式来准备自己的数据。

## 3.2 使用示例 DPO 数据集

ERNIE Kit 在 `examples/data/` 目录下提供了 `dpo-train.jsonl` 和 `dpo-eval.jsonl` 作为示例。我们可以直接使用这些数据来进行DPO训练。

**查看数据样本：**

In [8]:
import json

# 假设您在 ERNIE 项目的根目录下
dpo_train_file = "./examples/data/dpo-train.jsonl"

with open(dpo_train_file, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if i < 2:
            sample = json.loads(line.strip())
            print(json.dumps(sample, indent=4, ensure_ascii=False))
        else:
            break

{
    "src": [
        "请写一份关于支撑航空和飞行的物理基本原理的全面解释，包括升力、推力、重量和阻力等主题，以及伯努利定律、牛顿运动定律和空气性质等关键科学概念。使用清晰简洁的语言，并提供详细的例子和插图，以帮助读者理解这些概念。考虑塑造航空业的历史和技术发展，以及物理学在推动飞行能力方面发挥的作用。"
    ],
    "tgt": [],
    "response": [
        [
            "飞行物理学：航空原理全面指南\n\n**介绍**\n\n理解支撑航空和飞行的物理原理对于理解飞机在空中飞行的机制至关重要。在本指南中，我们将探讨飞行的基本因素，包括升力、推力、重量和阻力，并探索关键的科学概念，如伯努利定理、牛顿运动定律和空气的特性。通过使用清晰、简明的语言，并提供详细的例子和插图，本指南旨在使航空和飞行的概念对读者易于理解和吸引。\n\n**1. 飞行的四个力量**\n\n首先，认识到在飞行过程中作用于飞机的四个关键力量是很重要的：升力、推力、重量和阻力。这些力量之间的微妙平衡是飞机保持空中飞行和维持其所需飞行路径所必需的。\n\n1.1. 升力：升力是抵抗飞机重量并支撑其在空中的力量。升力是通过操纵飞机机翼周围的气压分布来产生的。机翼的形状，称为翼型，是为了优化升力而设计的。升力主要是通过伯努利定理和牛顿运动定律来解释的，我们稍后将详细探讨。\n\n1.2. 推力：推力是由飞机发动机产生的向前的力量，推动飞机穿过空气。喷气发动机、螺旋桨发动机甚至火箭发动机都可以提供必要的推力来克服阻力，这是另一个控制飞行的关键因素。\n\n1.3. 重量：重量是由于飞机质量所产生的重力。它向下作用并抵抗升力。为了保持水平飞行，飞机必须产生足够的升力来抵消其重量。\n\n1.4. 阻力：阻力是阻碍飞机在空气中运动的力量。它主要由两种类型组成：形状阻力，由于飞机的形状而产生，以及皮肤摩擦阻力，由于空气与飞机表面之间的摩擦而产生。最小化阻力对于实现高效飞行至关重要。\n\n**2. 伯努利定理和升力**\n\n伯努利定理是流体动力学中的基本概念，对于理解升力至关重要。该定理指出，当流体（在这种情况下是空气）的速度增加时，其压力降低。飞机机翼采用翼型形状，使空气在机翼上方移动得比下方快。这导致上表面的压力降低，下表面的压

这将打印出数据集中的前两个样本，帮助我们理解其具体结构和内容。

## 4. 开始 DPO 训练

现在，我们已经准备好了环境、模型和数据，是时候深入 DPO 训练的核心了。本章将首先从原理层面深入剖析 DPO，然后详细指导如何利用 ERNIE Kit 的强大功能来配置和启动训练任务。

### 4.1 DPO 原理深度剖析

理解 DPO 的工作原理，有助于我们更好地调整参数、分析结果。DPO 的核心思想是**将人类偏好直接转化为一个可供优化的损失函数**，从而绕过了 RLHF 中复杂的奖励模型训练和强化学习过程。

#### 4.1.1 从偏好到策略：DPO 的核心思想

1.  **隐式奖励模型**：DPO 巧妙地假设，存在一个未知的、理想的奖励模型 `r(x, y)`，它可以为任何一个“提示-回答”对 `(x, y)` 打分。人类的偏好数据（即哪个回答更好）反映了这个奖励模型的结果。具体来说，如果对于提示 `x`，回答 `y_w` (chosen) 优于 `y_l` (rejected)，那么我们认为 `r(x, y_w) > r(x, y_l)`。

2.  **Bradley-Terry 模型**：DPO 借用了 Bradley-Terry 模型来对这种偏好概率进行建模。该模型指出，人类偏好 `y_w` 超过 `y_l` 的概率 `p*(y_w > y_l | x)` 可以表示为：

    $\ p^*(y_w > y_l | x) = \frac{\exp(r(x, y_w))}{\exp(r(x, y_w)) + \exp(r(x, y_l))} = \sigma(r(x, y_w) - r(x, y_l)) \$ 
   

    其中，`σ` 是 Sigmoid 函数。这个公式直观地表示：两个回答的奖励分数之差越大，人类偏好其中一个的概率就越接近1。

3.  **连接语言模型与奖励**：接下来是 DPO 最为精妙的一步。它建立了一个语言模型策略 `π_θ`（即我们正在训练的模型）和上述隐式奖励模型 `r` 之间的解析关系。通过一系列推导，可以证明奖励函数可以表示为：

    $ \ r(x, y) = \beta \log \frac{\pi_\theta(y|x)}{\pi_{\text{ref}}(y|x)} \$ 

    这里：
    *   `π_θ(y|x)` 是当前策略模型生成回答 `y` 的概率。
    *   `π_ref(y|x)` 是参考模型（通常是SFT后的模型）生成回答 `y` 的概率。
    *   `β` 是一个超参数，用于控制策略模型与参考模型之间的散度（差异程度）。它平衡了“迎合偏好”和“保持语言能力”两个目标。

#### 4.1.2 DPO 损失函数：直观理解与数学表达

将上述奖励函数表达式代入 Bradley-Terry 模型，我们就可以用语言模型的概率来表示人类偏好概率。然后，通过最大化这个偏好概率的对数似然，我们最终得到了 DPO 的损失函数：

$-\mathbb{E}_{\left(x, y_w, y_l\right) \sim \mathcal{D}} \left[ \log \sigma\left( \beta \log \frac{\pi_\theta\left(y_w|x\right)}{\pi_{\text{ref}}\left(y_w|x\right)} - \beta \log \frac{\pi_\theta\left(y_l|x\right)}{\pi_{\text{ref}}\left(y_l|x\right)} \right) \right] = \mathcal{L}_{\text{DPO}}\left(\pi_\theta; \pi_{\text{ref}}\right)$


**直观理解这个复杂的公式：**

*   **核心目标**：损失函数的目标是最大化“模型对 `chosen` 回答的偏好”与“模型对 `rejected` 回答的偏好”之间的差距。
*   **偏好如何衡量**：模型对一个回答的“偏好”由 `log(π_θ / π_ref)` 这一项来衡量。它表示当前模型相对于参考模型，对这个回答的概率提升了多少。
*   **训练过程**：在每个训练步骤中，模型会调整参数 `θ`，使得 `log(π_θ(y_w|x) / π_ref(y_w|x))` 的值尽可能增大，同时使 `log(π_θ(y_l|x) / π_ref(y_l|x))` 的值尽可能减小。这样一来，两者的差值就会变大，`log σ(...)` 的值也随之增大，最终使得整个损失 `L_DPO` 减小。

**DPO vs. RLHF：为什么 DPO 更优越？**

*   **简单稳定**：DPO 将复杂的 RLHF 两阶段（奖励建模 + 强化学习）合并为一个简单的、带有新损失函数的分类任务。它不需要训练一个独立的奖励模型，也避免了强化学习中采样和训练不稳定的问题。
*   **数据高效**：DPO 直接利用偏好数据，不需要像 PPO 那样在训练中从策略模型大量采样，计算成本更低，数据利用率更高。
*   **效果相当甚至更好**：多项研究表明，DPO 能够在比 RLHF 更简单、成本更低的情况下，达到甚至超越 RLHF 的性能。

### 4.3 启动 DPO 训练

接下来，我们使用`erniekit`工具启动DPO训练。`erniekit`是ERNIE项目提供的一个便捷的命令行工具集，可以帮助我们轻松地管理模型的训练、评估和部署等任务。

**单卡训练命令：**

我们通过`--stage DPO`参数指定执行DPO训练任务。同时，我们还配置了`--model_name_or_path`（预训练模型路径）、`--train_dataset_path`（训练数据集）、`--eval_dataset_path`（评估数据集）和`--output_dir`（模型输出路径）等核心参数。此外，诸如`--max_seq_len`（最大序列长度）、`--learning_rate`（学习率）、`--max_steps`（最大训练步数）等超参数也被一并设置，以精确控制训练过程。

请注意：

根据调用栈，问 ernie/modeling_moe.py 将一个包含6个元素的 labels 元组传递给了 ernie/loss/dpo.py ，而后者期望的是5个元素。 score_deltas 这个多余的参数导致了错误。因此我们需要手动删除该参数

- 在/home/aistudio/ERNIE-develop/ernie/modeling_moe.py，大约（1709行）
- 为了快速演示，训练轮数我将减少

In [20]:
!erniekit train \
    --stage DPO \
    --model_name_or_path ../data/models/30656/ERNIE-4.5-0.3B-Paddle \
    --train_dataset_path ./examples/data/dpo-train.jsonl \
    --eval_dataset_path ./examples/data/dpo-eval.jsonl \
    --output_dir ./output/dpo_tutorial_checkpoint \
    --max_seq_len 8192 \
    --learning_rate 5.0e-7 \
    --warmup_steps 5 \
    --max_steps 10 \
    --save_steps 10 \
    --logging_steps 1 \
    --per_device_train_batch_size 1 \
    --gradient_accumulation_steps 36 \
    --bf16 True \
    --do_train \
    --fp16_opt_level O2

/opt/conda/envs/python35-paddle120-env/lib/python3.10/site-packages/paddle/utils/cpp_extension/extension_utils.py:715: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
/opt/conda/envs/python35-paddle120-env/lib/python3.10/site-packages/paddle/utils/cpp_extension/extension_utils.py:715: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
LAUNCH INFO 2025-07-18 12:04:39,160 -----------  Configuration  ----------------------
LAUNCH INFO 2025-07-18 12:04:39,160 auto_cluster_config: 0
LAUNCH INFO 2025-07-18 12:04:39,160 auto_parallel_config: None
LAUNCH INFO 2025-07-18 12:04:39,160 auto_tuner_json: None
LAUNCH INFO 2025-07-18 12:04:39

**监控训练过程：**

当训练开始后，您会在终端看到大量的日志输出。关注以下关键信息：

*   **`loss`**: 这是 DPO 损失的值。它应该随着训练的进行而稳步下降。
*   **`rewards/chosen` 和 `rewards/rejected`**: 这些是模型对 chosen 和 rejected 回答的平均隐式奖励得分。您应该会看到 `rewards/chosen` 的值逐渐增大，而 `rewards/rejected` 的值逐渐减小或保持较低水平。
*   **`rewards/accuracies`**: 这表示在一个批次中，模型正确地给予 `chosen` 回答比 `rejected` 回答更高奖励的样本比例。这个值应该趋向于 1.0。
*   **`rewards/margins`**: 这是 `rewards/chosen` 和 `rewards/rejected` 之间的平均差值。这个值越大，说明模型区分好坏回答的能力越强。
*   **`eval_loss`**: 在每个评估步骤 (`eval_steps`)，会在验证集上计算损失。观察这个值可以帮助您判断模型是否过拟合。

训练完成后，最终的模型权重和配置文件将保存在您 `output_dir` 指定的目录中（例如 `./output/ERNIE-4.5-0.3B-dpo`）。这个目录下的模型就是我们 DPO 成功的最终产物，它现在更懂得人类的偏好了。

## 5. 评估与推理

经过 DPO 训练，我们的模型在理论上已经更懂得人类的偏好。但“是骡子是马，拉出来遛遛”，本章将指导您如何科学地评估模型的效果，并将其部署到实际的交互式推理任务中，直观地感受其性能提升。

### 5.1 DPO 模型评估

DPO 的评估与传统的准确率、F1值等指标不同，它更侧重于**生成质量和偏好对齐度**的衡量。评估的核心是判断模型在面对相同的提示时，其生成的内容是否比 SFT 模型更符合我们的期望（例如，更安全、更详细、更具创造性等）。

#### 5.1.1 评估策略

1.  **构建高质量评估集**：
    *   设计一个专门的、包含各种挑战性提示的评估集至关重要。这些提示应该能够激发模型在不同维度上的能力，例如：
        *   **安全性**：包含一些可能诱导模型产生不安全或有偏见回答的边缘问题。
        *   **指令遵循**：设计带有复杂、多步骤或约束性指令的提示。
        *   **创造性与开放性问题**：提出需要模型发挥想象力或进行深入思考的问题。
        *   **事实性与知识性**：包含需要准确知识储备才能回答的问题。
    *   这个评估集不应该在训练或验证数据中出现过，以保证评估的公正性。

2.  **对比评估 (A/B Test)** ：
    *   最有效的评估方法是进行**头对头 (Head-to-Head)** 的比较。对于评估集中的每一个提示，分别使用 **SFT 模型** 和 **DPO 模型** 生成回答。
    *   将两个模型生成的回答（匿名，顺序打乱）呈现给人类评估者，让他们判断哪个回答更好，或者两者相当/都差。
    *   收集大量的评估结果，统计 DPO 模型的胜率、平局率和败率，从而量化其相对于 SFT 模型的提升。

3.  **自动化评估 (使用更强的模型作为裁判)** ：
    *   在没有足够人力进行大规模人工评估时，可以借助更强大的模型（如 GPT-4, ERNIE-4.0）作为“裁判”。
    *   设计一个裁判提示模板，将用户提示、SFT模型的回答、DPO模型的回答一并输入给裁判模型，让它判断哪个更好并给出理由。
    *   虽然自动化评估无法完全替代人类，但它提供了一种可扩展的、成本较低的评估方式，能够快速获得模型性能的初步印象。

## 6. 总结与展望

恭喜您！至此，您已经跟随本教程完整地走完了使用 ERNIE Kit 对 ERNIE-4.5-0.3B 模型进行直接偏好优化 (DPO) 的全过程。现在，让我们一起回顾这段旅程，并展望未来的可能性。

### 6.1 本教程核心内容回顾

在本教程中，我们系统性地学习并实践了以下核心内容：

1.  **DPO 核心思想**：我们深入理解了 DPO 如何巧妙地将人类偏好数据直接转化为一个简单的分类损失，从而绕过 RLHF 复杂的奖励建模和强化学习流程，实现了更高效、更稳定的模型对齐。

2.  **环境与代码准备**：我们搭建了基于 PaddlePaddle 和 ERNIE Kit 的开发环境，并学会了如何从源码安装 ERNIE Kit 以获取最新的功能和最大的灵活性。

3.  **数据准备与剖析**：我们掌握了 DPO 所需的 `(prompt, chosen, rejected)` 三元组数据格式，并使用 `UltraFeedback Binarized` 数据集作为示例，通过代码下载、解压并查看了数据样本，深刻理解了高质量偏好数据的关键要素。

4.  **DPO 训练配置与执行**：我们详细解读了 ERNIE Kit 的 DPO 配置文件，特别是对 `model_name_or_path` (必须是SFT模型)、`beta`、`learning_rate` 等关键超参数的理解和设置。我们还学会了如何通过命令行启动单卡和多卡训练，并如何解读训练日志中的关键指标来监控训练过程。

### 6.2 DPO 的局限性与未来展望

DPO 虽然强大，但并非银弹，它也有其局限性：

*   **对偏好数据质量的依赖**：DPO 的效果高度依赖于偏好数据的质量和一致性。标注模糊、有偏差或不一致的数据会严重误导模型的学习。
*   **Beta 值的敏感性**：超参数 `beta` 的选择对最终结果有显著影响，需要通过实验仔细调整。
*   **模式崩溃风险**：尽管比 RLHF 稳定，但在某些情况下，DPO 仍然可能导致模型为了过度迎合偏好而牺牲生成的多样性和流畅性。

**前沿的 DPO 变种与未来方向：**

学术界和工业界正在不断探索比 DPO 更优的对齐算法。当您对 DPO 有了深入的理解后，不妨关注以下一些前沿方向：

*   **IPO (Identity Preference Optimisation)** ：一种 DPO 的变体，通过修改损失函数来声称可以更好地防止模型在偏好数据上过拟合，ERNIE Kit 已支持此损失。
*   **KTO (Kahneman-Tversky Optimisation)** ：受人类决策理论启发，KTO 允许使用“好”和“坏”的单一样本进行对齐，而不再严格要求成对的偏好数据，极大地降低了数据标注的门槛。
*   **SimPO (Simple Preference Optimisation)** ：一种更简洁的损失函数设计，旨在提高训练效率和最终性能。

这些新的算法正在不断推动着大模型对齐技术的发展，也预示着未来我们会有更多、更好用的工具来让 AI 更好地为人类服务。

### 6.3 结语

本教程为您打开了通往大模型偏好对齐世界的大门。真正的学习始于实践。我们强烈鼓励您：

*   **尝试不同的超参数**：调整 `beta`、`learning_rate` 等，观察它们对模型行为的影响。
*   **构建自己的数据集**：尝试在您自己的业务场景或感兴趣的领域，构建一个小型的偏好数据集，并用本教程的方法进行训练。
*   **探索其他模型**：将本教程的方法应用到 ERNIE Kit 支持的其他模型上。

感谢您跟随本教程的学习。愿您在探索大模型的道路上，不断创造出更智能、更可靠、更符合人类价值观的 AI。祝您探索愉快！

# 问题反馈/与我联系： Wechat：G_Fuji